# Political Violence Targeting Women & Demonstrations Featuring Women

#### All violence targeting women, as well as demonstrations featuring women, are included in the data file below. The data in this file cover all events in which women were specifically targeted by political violence, not all events involving women in any way; the file also covers all demonstration events in which women were specifically featured, not all demonstrations involving women. 

#### This dataset was extract from ACLED Access Portal and we perform EDA using Python.


### Data collection

We import pandas and necessary libraries for dataframe manipulation.

In [9]:
import pandas as pd

In [10]:
df_gpv = pd.read_csv("gender_Sep27-1.csv")
display(df_gpv.shape)

(75563, 31)

In [11]:
df_gpv.head()

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp
0,BRA85924,2024-09-27,2024,1,Political violence,Violence against civilians,Sexual violence,Unidentified Gang (Brazil),NaN,Political militia,...,Sorriso,-12.5453,-55.7114,1,Gazeta Digital,Subnational,"On 27 September 2024, in Sorriso (Mato Grosso)...",0,NaN,1727730463
1,ECU3988,2024-09-27,2024,2,Political violence,Violence against civilians,Attack,Unidentified Armed Group (Ecuador),NaN,Political militia,...,Guayaquil,-2.1962,-79.8862,1,Ecuavisa; El Universo (Ecuador),National,"Around 27 September 2024 (as reported), in Gua...",1,women targeted: government officials,1727730464
2,MEX93498,2024-09-27,2024,2,Political violence,Violence against civilians,Abduction/forced disappearance,Unidentified Gang (Mexico),NaN,Political militia,...,Tijuana,32.5325,-117.0190,1,AFN - Tijuana,Subnational,"Around 27 September 2024 (as reported), in Tij...",0,NaN,1727730464
3,MEX93502,2024-09-27,2024,1,Political violence,Violence against civilians,Attack,Unidentified Gang (Mexico),NaN,Political militia,...,Ciudad de Mexico - Gustavo A. Madero,19.4829,-99.1135,1,Milenio,National,"On 27 September 2024, in Ciudad de Mexico - Gu...",1,NaN,1727730464
4,YEM92022,2024-09-27,2024,1,Political violence,Violence against civilians,Attack,Police Forces of Yemen (2017-) Houthi,Teachers (Yemen),State forces,...,Al Yahari,13.9966,44.0604,1,Khabar News Agency (Yemen),National,"On 27 September 2024, a Houthi-affiliated fema...",0,NaN,1727730471


In [8]:
df_gpv.keys()

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp'],
      dtype='object')

In [13]:
df_gpv[
    'geo_precision'
]

0        1
1        1
2        1
3        1
4        1
        ..
75558    1
75559    1
75560    2
75561    1
75562    1
Name: geo_precision, Length: 75563, dtype: int64